In [2]:
import duckdb

In [156]:
duckdb.sql("""
create or replace table Dirty as select row_number() over () as id, * from read_csv('Battery_List_Data_ADA.2026-02-11.csv');
describe Dirty;
""")

┌────────────────────────────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│                      column_name                       │ column_type │  null   │   key   │ default │  extra  │
│                        varchar                         │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────────────────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ id                                                     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Manufacturer Name                                      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Brand1                                                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Model Number                                           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Technology                                             │ VARCHAR     │ YES     │ NULL    │ NUL

In [158]:
duckdb.sql("""
create or replace table EntityCode as select * from read_csv('../EntityCodes.csv');
describe EntityCode;
""")

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name  │ column_type │  null   │   key   │ default │  extra  │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ProdMfr_Value │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Email_Value   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ EntityCode    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└───────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

### ProdMfr

In [159]:
ob_prod_mfr = duckdb.sql("""
select id, "Manufacturer Name" as ProdMfr_Value from Dirty;
""")
ob_prod_mfr.show(max_width=120)

┌───────┬────────────────────────────────────────────────────┐
│  id   │                   ProdMfr_Value                    │
│ int64 │                      varchar                       │
├───────┼────────────────────────────────────────────────────┤
│     1 │ Alpha ESS Co., Ltd.                                │
│     2 │ Altenergy Power System Inc.                        │
│     3 │ Altenergy Power System Inc.                        │
│     4 │ Altenergy Power System Inc.                        │
│     5 │ Altenergy Power System Inc.                        │
│     6 │ Altenergy Power System Inc.                        │
│     7 │ Altenergy Power System Inc.                        │
│     8 │ Altenergy Power System Inc.                        │
│     9 │ Altenergy Power System Inc.                        │
│    10 │ Altenergy Power System Inc.                        │
│     · │              ·                                     │
│     · │              ·                               

### ProdCode
If build the ProdCode, we first need to match the CEC manufacturer names to those given by SunSpec.

In [160]:
duckdb.sql("""
select
    *
from ob_prod_mfr
where ProdMfr_Value not in (select ProdMfr_Value from EntityCode)
""")

┌───────┬────────────────────────────────────────────────────┐
│  id   │                   ProdMfr_Value                    │
│ int64 │                      varchar                       │
├───────┼────────────────────────────────────────────────────┤
│     2 │ Altenergy Power System Inc.                        │
│     3 │ Altenergy Power System Inc.                        │
│     4 │ Altenergy Power System Inc.                        │
│     5 │ Altenergy Power System Inc.                        │
│     6 │ Altenergy Power System Inc.                        │
│     7 │ Altenergy Power System Inc.                        │
│     8 │ Altenergy Power System Inc.                        │
│     9 │ Altenergy Power System Inc.                        │
│    10 │ Altenergy Power System Inc.                        │
│    11 │ AltGrid Power LLC                                  │
│     · │         ·                                          │
│     · │         ·                                    

In [161]:
duckdb.sql("""
with cleaned_CEC_names as (
    select
        id,
        case 
            when ProdMfr_Value = 'Darfon Electronics Corp.' then 'Darfon Electronics Corporation'
            when ProdMfr_Value = 'Tesla Inc.' then 'Tesla'
            when ProdMfr_Value = 'Yotta Energy, Inc.' then 'YOTTA ENERGY INC'
            when ProdMfr_Value = 'GuangZhou MeriTech Power Co Ltd      ' then 'GuangZhou MeriTech Power Co Ltd'
            when ProdMfr_Value = 'Fortress Power LLC' then 'Fortress Power'
            when ProdMfr_Value = 'Holu Hou Energy LLC' then 'Holu Hou'
            when ProdMfr_Value = 'Discover Energy Systems Corp.' then 'Discover Energy Corp.'
            when ProdMfr_Value = 'Iron Edison Battery Company LLC' then 'Iron Edison LLC'
            when ProdMfr_Value = 'KORE Power, Inc.' then 'KORE Power'
            when ProdMfr_Value = 'LG Energy Solution, Ltd.' then 'LG Electronics Inc.'
            when ProdMfr_Value = 'Lunar Energy, Inc.' then 'Lunar Energy'
            when ProdMfr_Value = 'MidNite Solar, Inc.' then 'MidNite Solar Inc'
            when ProdMfr_Value = 'NINGBO DEYE ESS TECHNOLOGY CO., LTD' then 'Ningbo Deye Inverter Technology'
            when ProdMfr_Value = 'NeoVolta Inc.' then 'NeoVolta'
            when ProdMfr_Value = 'Outback Power LLC' then 'OutBack Power'
            when ProdMfr_Value = 'PANASONIC CORPORATION OF NORTH AMERICA' then 'Panasonic Corporation of North America'
            when ProdMfr_Value = 'Schneider Electric USA, Inc.' then 'Schneider Electric (US)'
            when ProdMfr_Value = 'Shenzhen Growatt New Energy Co., Ltd.' then 'Shenzhen Growatt New Energy Technology Co., Ltd'
            when ProdMfr_Value = 'SolaX Power Network Technology (Zhe jiang) Co., Ltd.' then 'Solax Power Network Technology (Zhejiang) Co., Ltd'
            when ProdMfr_Value = 'SimpliPhi Power, Inc.' then 'Simpliphi Power'
            when ProdMfr_Value = 'SolarEdge Technologies Ltd.' then 'SolarEdge Technologies Inc'
            else ProdMfr_Value
        end as ProdMfr_Value,
    from ob_prod_mfr
)
select
    *
from cleaned_CEC_names
where ProdMfr_Value not in (select ProdMfr_Value from EntityCode)
--  select distinct
--      fragment.ProdMfr_Value as CEC_ProdMfr,
--      EntityCode.ProdMfr_Value as SunSpec_ProdMfr,
--      EntityCode.EntityCode,
--  from EntityCode join (
--      select
--          id,
--          ProdMfr_Value,
--          split(ProdMfr_Value, ' ')[1] as F,
--      from cleaned_CEC_names
--      where ProdMfr_Value not in (select ProdMfr_Value from EntityCode)
--  ) as fragment on
--      EntityCode.ProdMfr_Value ilike format('%{}%', fragment.F)
--  order by CEC_ProdMfr, SunSpec_ProdMfr
""").pl().write_csv('unmatched.csv')

### Description

In [162]:
ob_description = duckdb.sql("""
select id, Description as Description_Value from Dirty;
""")
ob_description.show(max_width=120)

┌───────┬─────────────────────────────────────────────────────┐
│  id   │                  Description_Value                  │
│ int64 │                       varchar                       │
├───────┼─────────────────────────────────────────────────────┤
│     1 │ 9.8 kW, 8.2kWh lithium iron battery                 │
│     2 │ 5 kW, 5.76 kWh lithium iron phosphate battery       │
│     3 │ 6.14 kW, 10.24 kWh, lithium iron phosphate battery  │
│     4 │ 12.28 kW, 20.48 kWh, lithium iron phosphate battery │
│     5 │ 18.42 kW, 30.72 kWh, lithium iron phosphate battery │
│     6 │ 24.56 kW, 40.96 kWh, lithium iron phosphate battery │
│     7 │ 30.7 kW, 51.2 kWh, lithium iron phosphate battery   │
│     8 │ 36.84 kW, 61.44 kWh, lithium iron phosphate battery │
│     9 │ 42.98 kW, 71.68 kWh, lithium iron phosphate battery │
│    10 │ 49.12 kW, 81.92 kWh, lithium iron phosphate battery │
│     · │                       ·                             │
│     · │                       ·       

### BatteryChemistryType
Below are the BatteryChemistryTypes reported by the CEC.

In [163]:
duckdb.sql("""
select distinct Technology as BatteryChemistryType_Value from Dirty;
""")

┌────────────────────────────┐
│ BatteryChemistryType_Value │
│          varchar           │
├────────────────────────────┤
│ Lithium Iron               │
│ Lithium Ion Phosphate      │
│ Lithium Ion                │
│ Lead Acid                  │
│ Lithium Iron Phosphate     │
└────────────────────────────┘

For "Lead Acid", the product description does not provide additional information, so we do not change it to something more specific.

In [169]:
duckdb.sql("""
select distinct
    "Manufacturer Name",
    Technology as BatteryChemistryType_Value,
    Description_Value,
from Dirty join ob_description using (id)
where BatteryChemistryType_Value = 'Lead Acid'
""").show(max_width=120)

┌─────────────────────────────┬────────────────────────────┬───────────────────────────────────────┐
│      Manufacturer Name      │ BatteryChemistryType_Value │           Description_Value           │
│           varchar           │          varchar           │                varchar                │
├─────────────────────────────┼────────────────────────────┼───────────────────────────────────────┤
│ BAE Batteries USA           │ Lead Acid                  │ 1.91 kW, 1.67 kWh lead acid battery   │
│ BAE Batteries USA           │ Lead Acid                  │ 3.76 kW, 3.54 kWh lead acid battery   │
│ East Penn Manufacturing Co. │ Lead Acid                  │ 6.52 kW, 0.54 kWh, lead acid battery  │
│ East Penn Manufacturing Co. │ Lead Acid                  │ 0.209 kW, 1.68 kWh, lead acid battery │
│ East Penn Manufacturing Co. │ Lead Acid                  │ 0.276 kW, 2.22 kWh, lead acid battery │
│ BAE Batteries USA           │ Lead Acid                  │ 1.49 kW, 1.19 kWh lead acid ba

Note that the CEC data contains "Lithium Ion Phosphate" and "Lithium Iron", which appear to be typos.
We can verify that this is a typo by checking the descriptions of the associated batteries.

In [170]:
duckdb.sql("""
select distinct
    Technology as BatteryChemistryType_Value,
    'ion' in lower(Description_Value) <> 'iron' in lower(Description_Value) as should_be_iron,
    Description_Value,
from Dirty join ob_description using (id)
where BatteryChemistryType_Value = 'Lithium Ion Phosphate'
""").show(max_width=120)

┌──────────────────────┬────────────────┬──────────────────────────────────────────────────────────────────────────────┐
│ BatteryChemistryTy…  │ should_be_iron │                              Description_Value                               │
│       varchar        │    boolean     │                                   varchar                                    │
├──────────────────────┼────────────────┼──────────────────────────────────────────────────────────────────────────────┤
│ Lithium Ion Phosph…  │ true           │ 13.5 kWh DC energy storage system expansion pack, where XX are numbers and…  │
│ Lithium Ion Phosph…  │ true           │ 17.92 kW, 35.84 kWh lithium iron phosphate battery                           │
│ Lithium Ion Phosph…  │ true           │ 33.28 kW, 66.56 kWh lithium iron phosphate battery                           │
│ Lithium Ion Phosph…  │ true           │ 15.36 kW, 30.72 kWh lithium iron phosphate battery                           │
│ Lithium Ion Phosph…  │ true   

For "Lithium Iron", a google search suggests that a lithium iron battery without phospate does not exist, so we conclude that "Lithium Iron" should be "Lithium Iron Phosphate".

In [171]:
duckdb.sql("""
select distinct
    "Manufacturer Name",
    Technology as BatteryChemistryType_Value,
    Description_Value,
from Dirty join ob_description using (id)
where BatteryChemistryType_Value = 'Lithium Iron'
""").show(max_width=120)

┌──────────────────────────────────────────┬────────────────────────────┬──────────────────────────────────────────────┐
│            Manufacturer Name             │ BatteryChemistryType_Value │              Description_Value               │
│                 varchar                  │          varchar           │                   varchar                    │
├──────────────────────────────────────────┼────────────────────────────┼──────────────────────────────────────────────┤
│ Qcells North America                     │ Lithium Iron               │ 5.5kW, 10kWh, lithium iron battery system    │
│ SolaX Power Network Technology (Zhe ji…  │ Lithium Iron               │ 11.1 kW, 20 kWh, lithium iron battery system │
│ Qcells North America                     │ Lithium Iron               │ 11.1kW, 20kWh, lithium iron battery system   │
│ Qcells North America                     │ Lithium Iron               │ 8.3kW, 15kWh, lithium iron battery system    │
│ SolaX Power Network Technology

Now, we fix the inconsistencies and convert to Orange Button enumerations.

In [176]:
ob_battery_chemistry_type = duckdb.sql("""
select
    id,
    case
        when Technology = 'Lead Acid' then 'LeadAcid'
        when Technology in ('Lithium Iron Phosphate', 'Lithium Ion Phosphate', 'Lithium Iron') then 'LiFePO4'
        when Technology = 'Lithium Ion' then 'LiIon'
    end as BatteryChemistryType_Value
from Dirty;
""")
duckdb.sql("""
select distinct BatteryChemistryType_Value from ob_battery_chemistry_type
""")

┌────────────────────────────┐
│ BatteryChemistryType_Value │
│          varchar           │
├────────────────────────────┤
│ LiIon                      │
│ LiFePO4                    │
│ LeadAcid                   │
└────────────────────────────┘

### ProdCertification.CertificationAgencyName
Below we see some multiple labels for the same certification agency:
- CSA rebranded to CSA Group in 2012.
- "TUV Rhainland of North America" is a mispelling of "TUV Rheinland of North America"

In [177]:
duckdb.sql("""
select distinct "UL 1973 Certification Certifying Entity" CertificationAgencyName_Value from Dirty;
""")

┌────────────────────────────────┐
│ CertificationAgencyName_Value  │
│            varchar             │
├────────────────────────────────┤
│ UL                             │
│ TUV SUD                        │
│ TUV                            │
│ CSA                            │
│ SGS                            │
│ TUV SUD America                │
│ TUV Rheinland of North America │
│ CSA Group                      │
│ Intertek                       │
│ TUV Rhainland of North America │
│ TUV Rheinland                  │
├────────────────────────────────┤
│            11 rows             │
└────────────────────────────────┘

Now we normalize the labels.

In [178]:
ob_certification_agency_name = duckdb.sql("""
select
    id,
    case "UL 1973 Certification Certifying Entity"
        when 'CSA' then 'CSA Group'
        when 'TUV Rhainland of North America' then 'TUV Rheinland of North America'
        else "UL 1973 Certification Certifying Entity"
    end as CertificationAgencyName_Value
from Dirty;
""")
duckdb.sql("""
select distinct CertificationAgencyName_Value from ob_certification_agency_name;
""")

┌────────────────────────────────┐
│ CertificationAgencyName_Value  │
│            varchar             │
├────────────────────────────────┤
│ UL                             │
│ TUV SUD                        │
│ TUV                            │
│ SGS                            │
│ Intertek                       │
│ CSA Group                      │
│ TUV Rheinland of North America │
│ TUV SUD America                │
│ TUV Rheinland                  │
└────────────────────────────────┘

### ProdCertification.CertificationDate

In [179]:
ob_certification_date = duckdb.sql("""
select "UL 1973 Certification Certificate Date" as CertificationDate_Value from Dirty;
""")
ob_certification_date

┌─────────────────────────┐
│ CertificationDate_Value │
│          date           │
├─────────────────────────┤
│ 2021-06-02              │
│ 2022-02-02              │
│ 2025-12-29              │
│ 2025-12-29              │
│ 2025-12-29              │
│ 2025-12-29              │
│ 2025-12-29              │
│ 2025-12-29              │
│ 2025-12-29              │
│ 2025-12-29              │
│     ·                   │
│     ·                   │
│     ·                   │
│ 2023-03-10              │
│ 2024-06-20              │
│ 2022-11-07              │
│ 2024-09-09              │
│ 2024-09-06              │
│ 2024-09-09              │
│ 2023-01-05              │
│ 2023-05-12              │
│ 2023-05-12              │
│ 2024-11-27              │
├─────────────────────────┤
│   863 rows (20 shown)   │
└─────────────────────────┘

### ProdCertification.CertificationStandard
Here we see inconsistent formatting and typos:
- "Ed 3: 2022" vs "Ed. 3 : 2022"
- "Ed. 3 : 2023" should be "Ed. 3 : 2022"

In [180]:
duckdb.sql("""
select distinct "UL 1973 Certification Edition of UL 1973" as CertificationStandard_Value from Dirty;
""")

┌─────────────────────────────┐
│ CertificationStandard_Value │
│           varchar           │
├─────────────────────────────┤
│ Ed. 2 : 2018                │
│ Ed. 3 : 2022                │
│ Ed 3: 2022                  │
│ Ed. 3 : 2023                │
└─────────────────────────────┘

Now we fix these inconsistencies and convert to the Orange Button enumerations.

In [190]:
ob_certification_standard = duckdb.sql("""
select
    id,
    case "UL 1973 Certification Edition of UL 1973"
        when 'Ed. 2 : 2018' then 'UL1973_2_2018'
        when 'Ed 3: 2022' then 'UL1973_3_2022'
        when 'Ed. 3 : 2022' then 'UL1973_3_2022'
        when 'Ed. 3 : 2023' then 'UL1973_3_2022'
    end as CertificationStandard_Value from Dirty;
""")
duckdb.sql("""
select distinct CertificationStandard_Value from ob_certification_standard
""")

┌─────────────────────────────┐
│ CertificationStandard_Value │
│           varchar           │
├─────────────────────────────┤
│ UL1973_3_2022               │
│ UL1973_2_2018               │
└─────────────────────────────┘

### EnergyCapacityNominal

In [191]:
ob_energy_capacity_nominal = duckdb.sql("""
select
    id,
    'kWh' as EnergyCapacityNominal_Unit,
    "Nameplate Energy Capacity (kWh)" as EnergyCapacityNominal_Value,
from Dirty;
""")
ob_energy_capacity_nominal

┌───────┬────────────────────────────┬─────────────────────────────┐
│  id   │ EnergyCapacityNominal_Unit │ EnergyCapacityNominal_Value │
│ int64 │          varchar           │           double            │
├───────┼────────────────────────────┼─────────────────────────────┤
│     1 │ kWh                        │                         8.2 │
│     2 │ kWh                        │                        5.76 │
│     3 │ kWh                        │                       10.24 │
│     4 │ kWh                        │                       20.48 │
│     5 │ kWh                        │                       30.72 │
│     6 │ kWh                        │                       40.96 │
│     7 │ kWh                        │                        51.2 │
│     8 │ kWh                        │                       61.44 │
│     9 │ kWh                        │                       71.68 │
│    10 │ kWh                        │                       81.92 │
│     · │  ·                      

### DCOutput.PowerDCContinuousMax

In [192]:
ob_power_dc_continuous_max = duckdb.sql("""
select
    id,
    'kW' as PowerDCContinuousMax_Unit,
    "Nameplate Energy Capacity (kWh)" as PowerDCContinuousMax_Value,
from Dirty;
""")
ob_power_dc_continuous_max

┌───────┬───────────────────────────┬────────────────────────────┐
│  id   │ PowerDCContinuousMax_Unit │ PowerDCContinuousMax_Value │
│ int64 │          varchar          │           double           │
├───────┼───────────────────────────┼────────────────────────────┤
│     1 │ kW                        │                        8.2 │
│     2 │ kW                        │                       5.76 │
│     3 │ kW                        │                      10.24 │
│     4 │ kW                        │                      20.48 │
│     5 │ kW                        │                      30.72 │
│     6 │ kW                        │                      40.96 │
│     7 │ kW                        │                       51.2 │
│     8 │ kW                        │                      61.44 │
│     9 │ kW                        │                      71.68 │
│    10 │ kW                        │                      81.92 │
│     · │ ·                         │                        ·

# Columns that are too hard.
Clean these later.

### EfficiencyBatteryRoundTrip

In [198]:
duckdb.sql("""
select distinct
    'decimal_percent' as EfficiencyBatteryRoundTrip_Unit,
    "Manufacturer Declared Roundtrip Efficiency (%, Ac-AC)1" as EfficiencyBatteryRoundTrip_Value,
from Dirty;
""").show(max_width=120)

┌──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┐
│ EfficiencyBatteryR…  │                               EfficiencyBatteryRoundTrip_Value                                │
│       varchar        │                                            varchar                                            │
├──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ decimal_percent      │ 89 [JKS-R-7P12-US]\n89 [JKS-R-11P12-US]                                                       │
│ decimal_percent      │ Inverter 18kW]                                                                                │
│ decimal_percent      │ 98 [Sol-Ark 5K]\n98 [Sol-Ark 8K]\n98 [Sol-Ark 12K]\n98 [Sol-Ark 15K]                          │
│ decimal_percent      │ 89 [Sanctuary 3 \nInverter 12kW]\n89 [Sanctuary 3 \nInverter 18kW]\n89 [Sanctuary 3 \nInver…  │
│ decimal_percent      │ 81.04 [

In [196]:
duckdb.sql("""
select distinct
    "Certified JA12 Control  Strategies1"
from Dirty;
""").show(max_width=120)

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                         Certified JA12 Control  Strategies1                                          │
│                                                       varchar                                                        │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Sol-Ark 5K - Basic, TOU, Advanced\nSol-Ark 8K - Basic, TOU, Advanced\nSol-Ark 12K - Basic, TOU, Advanced\nSol-Ark …  │
│ No Information Submitted                                                                                             │
│ Inverter 24kW - Basic, TOU, Advance DR                                                                               │
│ Multiplus & Multiplus II & MP-II GX 48/3000/35 - Basic, TOU, Advance DR\nMultiplus & Multiplus II & MP-II GX 48/50…  │
│ Basic, TOU, Advanced DR       

In [200]:
duckdb.sql("""
select distinct "Declaration for JA12 Submitted1" from Dirty;
""").show(max_width=120)

┌─────────────────────────────────┐
│ Declaration for JA12 Submitted1 │
│             varchar             │
├─────────────────────────────────┤
│ Y (2022)                        │
│ N                               │
└─────────────────────────────────┘



# Ignored columns

In [201]:
duckdb.sql("""
select distinct Notes from Dirty;
""").show(max_width=120)

┌───────────────────────────────────────────────┐
│                     Notes                     │
│                    varchar                    │
├───────────────────────────────────────────────┤
│ Formerly Stored Energy Inc, PF5-LFP14400-2A01 │
│ Formerly Stored Energy Inc, PF5-LFP33600-2A01 │
│ Formerly Stored Energy Inc, PF5-LFP09600-2A01 │
│ Formerly Stored Energy Inc, PF5-LFP28800-2A01 │
│ NULL                                          │
│ Formerly Stored Energy Inc, PF5-LFP19200-2A01 │
│ Formerly Stored Energy Inc, PF5-LFP38400-2A01 │
│                                               │
│ Formerly Stored Energy Inc, PF5-LFP24000-2A01 │
└───────────────────────────────────────────────┘



In [202]:
duckdb.sql("""
select distinct "CEC Listing Date" from Dirty;
""").show(max_width=120)

┌──────────────────┐
│ CEC Listing Date │
│       date       │
├──────────────────┤
│ 2024-11-12       │
│ 2019-09-27       │
│ 2021-10-21       │
│ 2024-08-12       │
│ 2024-10-01       │
│ 2023-10-11       │
│ 2025-09-02       │
│ 2023-12-01       │
│ 2023-11-01       │
│ 2025-12-11       │
│     ·            │
│     ·            │
│     ·            │
│ 2023-02-13       │
│ 2025-11-12       │
│ 2022-04-21       │
│ 2022-06-21       │
│ 2026-01-21       │
│ 2025-12-22       │
│ 2025-08-11       │
│ 2023-05-01       │
│ 2026-02-11       │
│ 2023-02-21       │
├──────────────────┤
│     121 rows     │
│    (20 shown)    │
└──────────────────┘



In [203]:
duckdb.sql("""
select distinct "Last Update" from Dirty;
""").show(max_width=120)

┌─────────────┐
│ Last Update │
│    date     │
├─────────────┤
│ 2025-12-11  │
│ 2023-12-01  │
│ 2023-07-03  │
│ 2025-08-21  │
│ 2024-04-02  │
│ 2023-05-11  │
│ 2024-09-11  │
│ 2026-02-02  │
│ NULL        │
│ 2024-06-03  │
│ 2024-11-01  │
│ 2024-01-02  │
│ 2024-03-01  │
│ 2023-05-22  │
│ 2019-12-23  │
│ 2023-09-21  │
├─────────────┤
│   16 rows   │
└─────────────┘



# Building clean data